In [1]:
import warnings
warnings.filterwarnings('ignore')
import re
import numpy as np

In [2]:
doc = open('../DataSets/text8','r').read().lower()

In [3]:
len(doc)

100000000

In [4]:
corpus_list = re.split('\W+', doc)

In [5]:
len(corpus_list)

17005208

In [6]:
cutOffValue = 100
from collections import defaultdict
frequency = defaultdict(int)
for token in corpus_list:
    frequency[token] += 1
processedCorpus_list = [token for token in corpus_list 
                        if frequency[token] >= cutOffValue]

In [7]:
len(processedCorpus_list)

15471435

In [8]:
len(corpus_list)-len(processedCorpus_list)

1533773

In [9]:
len(frequency)

253855

In [10]:
allWords = np.array(list(frequency.keys()))
allCounts = np.array(list(frequency.values()))

In [11]:
len(allWords)

253855

In [12]:
vocab = allWords[allCounts >= cutOffValue]
wordCounts = allCounts[allCounts >= cutOffValue]

In [13]:
len(vocab)

11815

In [14]:
from scipy.sparse import lil_matrix

In [15]:
def computeWordContextMatrix(corpus_list,vocab=None,windowSize=2):
    if vocab is None:
        vocab = sorted(list(set(cospus_list)))
    numWords = len(vocab)
    M = np.zeros((numWords,numWords))
    # M = lil_matrix((numWords,numWords)) #for computing big matrix by splitting into pieces
    W2I = dict(zip(vocab,np.arange(numWords)))
    I2W = dict(zip(np.arange(numWords),vocab))
    doc = corpus_list
    docLen = len(doc)
    curIdx = 0
    while curIdx < docLen:
        left = max(curIdx-windowSize,0)
        right = min(curIdx+windowSize+1,docLen)
        wordsInContext = doc[left:curIdx] + doc[curIdx+1:right]
        currentWord = doc[curIdx]
        currentWordIdx = W2I[currentWord]
        for word in wordsInContext:
            contextWordIdx = W2I[word]
            M[currentWordIdx,contextWordIdx] += 1
        curIdx += 1
    return M,W2I,I2W

In [16]:
M,W2I,I2W = computeWordContextMatrix(processedCorpus_list,vocab)

In [17]:
M.shape

(11815, 11815)

In [18]:
word = 'good'
print(W2I[word],I2W[190])

190 good


In [19]:
v = M[W2I['good'],:]
print(v)

[  0.   0. 214. ...   0.   0.   0.]


In [20]:
v.shape

(11815,)

In [21]:
def pmi(M, positive=True):
    col_totals = np.sum(M,axis=0)
    total = col_totals.sum()
    row_totals = np.sum(M,axis=1)
    expected = np.outer(row_totals, col_totals) / total
    M = M / expected    
    with np.errstate(divide='ignore'):
        M = np.log(M)
    M[np.isinf(M)] = 0.0  
    if positive:
        M[M < 0] = 0.0
    return M

In [22]:
M = pmi(M)

In [23]:
M[W2I['good'],:]

array([0.       , 0.       , 0.4605286, ..., 0.       , 0.       ,
       0.       ])

# Latent Semantic Analysis (LSA)

In [24]:
from sklearn.decomposition import TruncatedSVD, PCA, IncrementalPCA # IncrementalPCA is used when the matrix is very-very large

In [25]:
transformer = TruncatedSVD(n_components=100)  # new dimention size = 100

In [26]:
M_reduced = transformer.fit_transform(M)

In [27]:
print(M.shape)
print(M_reduced.shape)

(11815, 11815)
(11815, 100)


In [28]:
M_reduced[W2I['good'],:]

array([ 2.35845596e+01, -3.84230767e+00, -3.72714382e+00, -4.18267227e+00,
       -4.50620860e+00, -7.88498267e+00, -1.00250679e+00, -3.38179848e+00,
        5.66383617e-02, -2.34153819e+00,  7.71481499e+00,  2.24923434e+00,
       -2.86809213e+00,  3.30206685e+00,  5.34319894e-01, -1.45753630e-01,
       -6.91953826e+00,  4.47009982e+00, -8.77516881e-01, -3.41816223e-01,
        1.34011207e+00,  7.91244899e-01,  1.87614785e+00,  3.97524597e+00,
       -2.32709745e+00,  8.43940858e-01, -1.32130813e+00,  2.26466713e+00,
        1.38199081e+00,  1.32274875e-01, -4.03627737e+00,  3.90541555e+00,
       -4.48808236e-01,  4.52999645e+00,  7.98620985e-01,  1.36358028e+00,
       -1.10799523e+00, -3.11078507e+00,  1.79666804e+00, -2.51067837e+00,
       -3.60075139e-01, -1.86699839e+00, -1.32702197e+00,  2.59448927e+00,
       -1.20815749e+00, -1.90374782e+00,  1.35565159e+00,  1.72011108e+00,
       -2.90397363e-01,  9.35469145e-01,  3.23722582e+00,  1.38203420e+00,
       -3.38730214e-01,  

# Word Semantics

## Cosine Similarity

In [29]:
def getNorms(E):  # e = single row vector or multiple row matrix
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = np.sum(E**2,axis=1)**0.5  # calculate magnitude ( ||E|| )
    return nrms

In [30]:
v = M_reduced[W2I['good'],:]
print(v.shape)

v = v[np.newaxis,:]
print(v.shape)

print(getNorms(v))

(100,)
(1, 100)
[33.57804637]


In [31]:
v = M_reduced[[1,3,5],:]
print(v.shape)
print(getNorms(v))

(3, 100)
[20.38316302 39.12603251 34.56945578]


In [32]:
def normalize(E):  # e = single row vector or multiple row matrix
    if E.ndim == 1:
        E = E[np.newaxis,:]
    nrms = getNorms(E)
    return E/nrms[:,np.newaxis]

In [33]:
# v = M_reduced[1,:]
v = M_reduced[[1,3,5],:]
v2 = normalize(v)
print(getNorms(v2), getNorms(v)) # normalize makes it 1

[1. 1. 1.] [20.38316302 39.12603251 34.56945578]


In [34]:
# Measuring cosine similarity of 'v' with all the vectors of 'E'.

def cosineSimilarity(E,v): # here vector v is always a single vector; E can be single or multiple
    E = normalize(E)
    v = normalize(v)
    scores = E.dot(v.T)
    return scores

In [35]:
E = M_reduced[W2I['good'],:]
v = M_reduced[W2I['nice'],:]

print(cosineSimilarity(E,v))

[[0.62979723]]


In [36]:
E = M_reduced[[1,3,5],:]
v = M_reduced[W2I['nice'],:]

scores = cosineSimilarity(E,v)
print(scores)
print(scores.shape)

scores = scores.squeeze()
print(scores)
print(scores.shape)

[[0.44298945]
 [0.44520478]
 [0.30407804]]
(3, 1)
[0.44298945 0.44520478 0.30407804]
(3,)


In [37]:
E = M_reduced
v = M_reduced[W2I['nice'],:]

scores = cosineSimilarity(E,v)
scores = scores.squeeze()
print(scores)
print(scores.shape)

[0.30155857 0.44298945 0.28930459 ... 0.41516238 0.22871733 0.3565078 ]
(11815,)


In [38]:
def getMostSimilarWords(E, word, W2I, topn = 10):
    if type(word) is str:
        v = E[W2I[word],:]
    else:
        v = word
    scores = cosineSimilarity(E, v)
    scores = scores.squeeze()
    sortedScores = np.sort(scores)[::-1]
    sortedIdx = np.argsort(scores)[::-1]
    topNScores = sortedScores[:topn]
    topNScoreIdx = sortedIdx[:topn]
    return topNScores, topNScoreIdx

In [39]:
t10Scores, t10idx = getMostSimilarWords(M_reduced, 'nice', W2I)

In [40]:
print(t10Scores)
print(t10idx)

[1.         0.72842868 0.71886702 0.70862623 0.7038367  0.70305251
 0.69787531 0.6876513  0.6836204  0.68339808]
[ 1368  7548  3248  4441  9610  4375 10269  4631  3543  8678]


In [41]:
for s, i in zip(t10Scores, t10idx):
    print(s, I2W[i])

1.0 nice
0.7284286830108052 pretty
0.7188670165041214 happy
0.7086262325483703 stuff
0.7038367007691811 dirty
0.7030525050529061 likes
0.6978753121229505 welcome
0.6876512962680659 looks
0.6836204026605797 boys
0.6833980790175862 maybe
